In [1]:
# let's take one dataset for example
from multilab.datasets import reuter
sentences, labels = reuter()

[nltk_data] Downloading package reuters to /Users/monk/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [2]:
# importing preprocessing class
# let's play with that

from multilab.preprocess import Text_preprocessing
text_preprocessing = Text_preprocessing()

In [3]:
dataframe = text_preprocessing.labels_to_dataframe(sentences,labels)

In [4]:
preprocessded_dataset = text_preprocessing.initial_preprocess(dataframe, chunk_value = 25)

lower_case done
punctuation removed
text cleaning done


In [5]:
dataset, frequency_list = text_preprocessing.keep_labels(preprocessded_dataset,keep_ratio=0.10)
frequency_list

[('acq', 2369),
 ('crude', 578),
 ('earn', 3964),
 ('grain', 582),
 ('interest', 478),
 ('money-fx', 717),
 ('ship', 286),
 ('trade', 485),
 ('wheat', 283)]

In [6]:
slice_dataset = text_preprocessing.dataset_slice(dataset,ratio=0.25)


In [9]:
import numpy as np
all_sente = slice_dataset['text']
all_label = np.array(slice_dataset.drop('text', 1))

In [12]:
X_train, X_test, y_train, y_test = text_preprocessing.split_dataset(all_sente, all_label)


In [11]:
from multilab.models import Bilstm

config = {
            'vocab_size'                 : 7000,
            'no_of_labels'               : 9,
            'rnn_units'                  : 256, 
            'word_embedding_dim'         : 300, 
            'learning_rate'              : 0.001, 
            'pretrained_embedding_matrix': None,
            'dropout'                    : 0.2,
            'epoch'                      : 5,
            'batch_size'                 : 128,
            'result_path'                : '/Users/aaditya/Desktop',
            'last_output'                : False,
            'train_embedding'            : True
        }


bl = Bilstm(X_train, y_train, X_test,  y_test, config)

In [14]:
print(len(frequency_list))

9


In [14]:
X_train.iloc[0]

'frances balladur says target zone notion nearer french finance minister edouard balladur said that the financial community is closer to arriving at a system of'